## Visualize benchmark results in a table

In [1]:
import os
import numpy as np
import functools
import pandas as pd
from matplotlib import colors
import matplotlib.pyplot as plt
from collections import defaultdict
from lm_polygraph.utils.manager import UEManager, _delete_nans

def b_g(s, A, cmap='PuBu', low=0.8, high=0):
    # Pass the columns from Dataframe A
    i = A.columns.tolist().index(s.name)
    a = A.values[:,i].copy()
    if s.name[-1] in ['rcc-auc']:
        a = -a
    if s.name[0] == 'BARTScoreSeq-rh':
        a = -a
    rng = a.max() - a.min()
    norm = colors.Normalize(a.min() - (rng * low),
                        a.max() + (rng * high))
    normed = norm(a)
    c = [colors.rgb2hex(x) for x in plt.colormaps[cmap](normed)]
    return ['background-color: %s' % color for color in c]

def get_array(dfs, row, col):
    vals = []
    for df in dfs:
        if row in df.index and col in df.columns:
            vals.append(df.loc[row, col])
    return vals

def pretty_plot(dataset_name, man_files, except_metrics=[], except_gen=['BARTScoreSeq-rh'], level='sequence'):
    dfs = []
    if isinstance(dataset_name, str):
        dataset_name = [dataset_name]
        man_files = [man_files]
    columns = []
    for group_name, group_files in zip(dataset_name, man_files):
        gen_metrics = None
        for f in group_files:
            man = UEManager.load(f)
            # print(man.gen_metrics)
            # print({str(k) : v for k, v in man.gen_metrics})
            print({str(k) : np.mean(v) for k, v in man.gen_metrics.items()})
            estimators = [e for (l, e) in man.estimations.keys() if l == level]
            gen_metrics = list(set([(group_name, gen_name, m_name)
               for (l, e_name, gen_name, m_name) in man.metrics
               if l == level and (m_name not in except_metrics) and (gen_name not in except_gen)]))
            gen_metrics.sort()
            df = {k: {} for k in gen_metrics}
            for (l, e_name, gen_name, m_name), value in man.metrics.items():
                if l == level and (m_name not in except_metrics) and (gen_name not in except_gen):
                    df[group_name, gen_name, m_name][e_name] = value
            for k in gen_metrics:
                df[k] = [df[k][e] for e in estimators]
            df = pd.DataFrame(data=df, index=[e for e in estimators])
            df = df.reindex(columns=gen_metrics)
            dfs.append(df)
        print('Will measure variance using', len(group_files), 'seeds')
        columns += gen_metrics
    assert(len(dfs) > 0)
    index = dfs[0].index
    mean, total = defaultdict(lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int))
    for col in columns:
        for row in index:
            vals = get_array(dfs, row, col)
            mean[row][col] = -np.mean(vals)
            # total[row][col] = '{:.2f} ± {:.2f}'.format(np.mean(vals).item() * 100, np.std(vals).item() * 100)
            total[row][col] = '{:.2f}'.format(np.mean(vals).item() * 100)

    total_df = pd.DataFrame([[total[row][col] for col in columns] for row in index],
                            index=index, columns=pd.MultiIndex.from_tuples(columns))
    mean_df = pd.DataFrame([[mean[row][col] for col in columns] for row in index],
                           index=index, columns=pd.MultiIndex.from_tuples(columns))
    
    s = total_df.style.apply(functools.partial(b_g, A=mean_df, cmap='Greens'), axis=0)
    s.set_table_styles([{  # for row hover use <tr> instead of <td>
        'selector': 'td:hover',
        'props': [('background-color', '#ffffb3')]
    }, {
        'selector': '.index_name',
        'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
    }])
    s.set_table_styles({
        columns[i]: [{'selector': 'th', 'props': 'border-left: {}px solid black'.format(1 if columns[i][0] == columns[i - 1][0] else 2)},
                     {'selector': 'td', 'props': 'border-left: {}px solid black'.format(1 if columns[i][0] == columns[i - 1][0] else 2)}]
        for i in range(1, len(columns)) if i == 0 or columns[i][1] != columns[i - 1][1]
    }, overwrite=False, axis=0)
    return s

In [ ]:
/home/dzianis.piatrashyn/lm-polygraph/workdir/output/qa/{'path': 'meta-llama/Llama-3.1-8B', 'ensemble': False, 'mc': False, 'mc_seeds': None, 'dropout_rate': None, 'type': 'CausalLM', 'path_to_load_script': 'model/llama-3.1-8b.py', 'load_model_args': {'device_map': 'auto'}, 'load_tokenizer_args': {}}/['denis1699/gsm8k_reasoning']/2025-06-23/22-47-30

In [2]:
# visualize results in a table
s = pretty_plot(
    'gsm8k, Llama3.1-8b',
    # outputs generated by scripts/polygraph_eval benchmark
    # provide several seeds to calculate variance
    ["../workdir/output/qa/{'path': 'meta-llama/Llama-3.1-8B', 'ensemble': False, 'mc': False, 'mc_seeds': None, 'dropout_rate': None, 'type': 'CausalLM', 'path_to_load_script': 'model/llama-3.1-8b.py', 'load_model_args': {'device_map': 'auto'}, 'load_tokenizer_args': {}}/['denis1699/gsm8k_reasoning']/2025-06-23/22-47-30/ue_manager_seed1"])

{"('sequence', 'AccuracyReasoning')": 0.5049279757391963}
Will measure variance using 1 seeds


In [3]:
s

In [4]:
s.to_html("table_full_gsm8k.html")

In [4]:
man = UEManager.load("../workdir/output/qa/{'path': 'meta-llama/Llama-3.1-8B', 'ensemble': False, 'mc': False, 'mc_seeds': None, 'dropout_rate': None, 'type': 'CausalLM', 'path_to_load_script': 'model/llama-3.1-8b.py', 'load_model_args': {'device_map': 'auto'}, 'load_tokenizer_args': {}}/['denis1699/gsm8k_reasoning']/2025-06-23/22-47-30/ue_manager_seed1")

In [14]:
man.stats.keys()

dict_keys(['input_texts', 'target_texts', 'greedy_texts', 'greedy_tokens', 0, 1, 100])

In [7]:
for (gen_level, gen_name), generation_metric in man.gen_metrics.items():
    print(f"gen_level: {gen_level}, gen_name: {gen_name}")
    print(generation_metric)
    print(np.mean(generation_metric))

gen_level: sequence, gen_name: AccuracyReasoning
[0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,

In [6]:
for (gen_level, gen_name), generation_metric in man.gen_metrics.items():
    for (e_level, e_name), estimator_values in man.estimations.items():
        print(f"\nEstimator: {e_name}")
        ue, metric = _delete_nans(estimator_values, generation_metric)
        for prob, val in zip(ue, metric):
            print("Row:", prob, val)
        # print(ue)
        # print(metric)
        break


Estimator: MaximumSequenceProbability
Row: 45.69231033325195 0
Row: 3.195338249206543 1
Row: 41.15781021118164 0
Row: 25.3309326171875 1
Row: 34.2806396484375 0
Row: 33.592838287353516 1
Row: 11.664873123168945 1
Row: 37.473052978515625 0
Row: 56.45173263549805 0
Row: 38.068603515625 1
Row: 17.79863739013672 1
Row: 34.01557922363281 0
Row: 51.923309326171875 0
Row: 44.971534729003906 0
Row: 21.940759658813477 1
Row: 24.777555465698242 0
Row: 23.758705139160156 1
Row: 40.20061492919922 0
Row: 42.98601531982422 1
Row: 38.0367317199707 1
Row: 29.503767013549805 0
Row: 29.52762222290039 0
Row: 18.743728637695312 1
Row: 32.29069137573242 0
Row: 27.996191024780273 0
Row: 42.62298583984375 1
Row: 33.138126373291016 0
Row: 32.56157302856445 1
Row: 24.530540466308594 0
Row: 55.48281478881836 0
Row: 34.33433532714844 0
Row: 61.41001892089844 1
Row: 30.239742279052734 1
Row: 18.849552154541016 0
Row: 20.73053550720215 0
Row: 14.835147857666016 1
Row: 31.51729393005371 1
Row: 45.65056610107422 0


In [10]:
for (gen_level, gen_name), generation_metric in man.gen_metrics.items():
    print(f"Gen. metric: {gen_name}")
    for ue_metric in [PredictionRejectionArea(), PredictionRejectionArea(max_rejection=0.5), ROCAUC()]:
        print(f"UE metric: {ue_metric}")
        oracle_score_all = ue_metric(
            -np.array(generation_metric), np.array(generation_metric)
        )
        random_score_all = get_random_scores(
            ue_metric, np.array(generation_metric)
        )
        print(f"Oracle: {oracle_score_all}")
        print(f"Random: {random_score_all}")
        for (e_level, e_name), estimator_values in man.estimations.items():
            print(f"\nEstimator: {e_name}")
            ue, metric = _delete_nans(estimator_values, generation_metric)
            ue_metric_val = ue_metric(ue, metric)
            print(f"{ue_metric}: {ue_metric_val}")
            ue_metric_val_normalized = normalize_metric(ue_metric_val, oracle_score_all, random_score_all)
            print(f"{ue_metric}_normalized: {ue_metric_val_normalized}")
            if ue_metric_val_normalized < 0:
                ue_metric_val = ue_metric(ue, -metric)
                print(f"{ue_metric}: {ue_metric_val}")
                ue_metric_val_normalized = normalize_metric(ue_metric_val, oracle_score_all, random_score_all)
                print(f"{ue_metric}_normalized: {ue_metric_val_normalized}")
        print("\n")

Gen. metric: AccuracyReasoning
UE metric: prr
Oracle: 0.8440860896550976
Random: 0.5008389588121596

Estimator: MaximumSequenceProbability
prr: 0.6091520515024208
prr_normalized: 0.31555425510555196

Estimator: Perplexity
prr: 0.46759089915279034
prr_normalized: -0.09686332869766294
prr: 0.5324091008472096
prr_normalized: 0.09197496263849562

Estimator: MeanTokenEntropy
prr: 0.6285632531311238
prr_normalized: 0.372105934302501

Estimator: ProbasMinWithCoT
prr: 0.5115382306319507
prr_normalized: 0.03117075383417225

Estimator: StepsMinProb
prr: 0.4886954766871503
prr_normalized: -0.03537824801386568
prr: 0.5113045233128497
prr_normalized: 0.030489881954698525

Estimator: StepsAvgProb
prr: 0.5264580253066304
prr_normalized: 0.07463737987133699

Estimator: StepsMaxProb
prr: 0.5788290104559803
prr_normalized: 0.2272125376614065

Estimator: Step2QuestionNLI_entail_mean
prr: 0.5025693181021734
prr_normalized: 0.005041147134324142

Estimator: Step2QuestionNLI_entail_min
prr: 0.514815978920090

In [6]:
man = UEManager.load("../workdir/output/qa/{'path': 'meta-llama/Llama-3.1-8b', 'ensemble': False, 'mc': False, 'mc_seeds': None, 'dropout_rate': None, 'type': 'CausalLM', 'path_to_load_script': 'model/llama-3.1-8b-quantized.py', 'load_model_args': {'device_map': 'auto'}, 'load_tokenizer_args': {}}/['denis1699/gsm8k_reasoning']/2025-06-02/09-01-59/ue_manager_seed1")

In [8]:
print(man.estimations.keys())

dict_keys([('sequence', 'MaximumSequenceProbability'), ('sequence', 'Perplexity'), ('sequence', 'MeanTokenEntropy'), ('sequence', 'ProbasMinWithCoT')])


In [4]:
for (gen_level, gen_name), generation_metric in man.gen_metrics.items():
    print(f"gen_level: {gen_level}, gen_name: {gen_name}")
    print(generation_metric)

gen_level: sequence, gen_name: Accuracy
[1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]
gen_level: sequence, gen_name: AccuracyReasoning
[1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1]


In [8]:
man

In [12]:
from lm_polygraph.ue_metrics import *
from lm_polygraph.ue_metrics.ue_metric import (
    get_random_scores,
    normalize_metric,
)

In [36]:
for (gen_level, gen_name), generation_metric in man.gen_metrics.items():
    print(f"Gen. metric: {gen_name}")
    for ue_metric in [PredictionRejectionArea(), PredictionRejectionArea(max_rejection=0.5), ROCAUC()]:
        print(f"UE metric: {ue_metric}")
        oracle_score_all = ue_metric(
            -np.array(generation_metric), np.array(generation_metric)
        )
        random_score_all = get_random_scores(
            ue_metric, np.array(generation_metric)
        )
        print(f"Oracle: {oracle_score_all}")
        print(f"Random: {random_score_all}")
        for (e_level, e_name), estimator_values in man.estimations.items():
            print(f"\nEstimator: {e_name}")
            ue, metric = _delete_nans(estimator_values, generation_metric)
            ue_metric_val = ue_metric(ue, metric)
            print(f"{ue_metric}: {ue_metric_val}")
            ue_metric_val_normalized = normalize_metric(ue_metric_val, oracle_score_all, random_score_all)
            print(f"{ue_metric}_normalized: {ue_metric_val_normalized}")
        print("\n")

Gen. metric: Accuracy
UE metric: prr
Oracle: 0.8370043678619956
Random: 0.48821631197200654

Estimator: MaximumSequenceProbability
prr: 0.5137612488451946
prr_normalized: 0.07323913890344097

Estimator: Perplexity
prr: 0.5128699611662411
prr_normalized: 0.07068375415358404

Estimator: MeanTokenEntropy
prr: 0.5360746893352977
prr_normalized: 0.13721334935387264

Estimator: ProbasMinWithCoT
prr: 0.4905028917755164
prr_normalized: 0.00655578585589837


UE metric: prr_0.5
Oracle: 0.6744087357239913
Random: 0.4891973259163078

Estimator: MaximumSequenceProbability
prr_0.5: 0.4802610989516802
prr_0.5_normalized: -0.04824879295453065

Estimator: Perplexity
prr_0.5: 0.487126467180894
prr_0.5_normalized: -0.011181053789095128

Estimator: MeanTokenEntropy
prr_0.5: 0.49878368845074994
prr_0.5_normalized: 0.05175902793675742

Estimator: ProbasMinWithCoT
prr_0.5: 0.5155070432897816
prr_0.5_normalized: 0.14205235736174557


UE metric: roc-auc
Oracle: 0.0
Random: 0.5026202480992397

Estimator: Maximu

In [ ]:
for (gen_level, gen_name), generation_metric in man.gen_metrics.items():
    for ue_metric in man.ue_metrics:
        log.info(f"Metric: {ue_metric}")

        oracle_score_all = ue_metric(
            -np.array(generation_metric), np.array(generation_metric)
        )
        random_score_all = get_random_scores(
            ue_metric, np.array(generation_metric)
        )
        for (e_level, e_name), estimator_values in self.estimations.items():
            if gen_level != e_level:
                continue
            if len(estimator_values) != len(generation_metric):
                raise Exception(
                    f"Got different number of metrics for {e_name} and {gen_name}: "
                    f"{len(estimator_values)} and {len(generation_metric)}"
                )

            n_nans = np.sum(~np.isfinite(estimator_values))
            if n_nans > 0:
                log.warning(f"We got {n_nans} nans in {e_name} estimator.")

            n_nans = np.sum(~np.isfinite(generation_metric))
            if n_nans > 0:
                log.warning(
                    f"We got {n_nans} nans in {gen_name} generation metric."
                )

            ue, metric = _delete_nans(estimator_values, generation_metric)
            if len(ue) == 0:
                self.metrics[e_level, e_name, gen_name, str(ue_metric)] = np.nan
            else:
                if len(ue) != len(estimator_values):
                    oracle_score = ue_metric(-metric, metric)
                    random_score = get_random_scores(ue_metric, metric)
                else:
                    oracle_score = oracle_score_all
                    random_score = random_score_all

                ue_metric_val = ue_metric(ue, metric)
                self.metrics[e_level, e_name, gen_name, str(ue_metric)] = (
                    ue_metric_val
                )
                self.metrics[
                    e_level, e_name, gen_name, str(ue_metric) + "_normalized"
                ] = normalize_metric(ue_metric_val, oracle_score, random_score)